In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

def pos_changes(csv_file_path):
    
    df = pd.read_csv(csv_file_path)
    df.columns = ['pos', 'time']
    changes = []

    previous_value = None
    for index, row in df.iterrows():
        current_value = row['pos']
        current_time = row['time']
        if previous_value is None or current_value != previous_value:
            changes.append([current_value, current_time])
        previous_value = current_value
        
    return changes



def add_constant(data, constant):
    for row in data:
        row[1] += constant
    return data


def generate_rate_data_by_TR(file_path):
    # 读取Excel文件
    df = pd.read_csv(file_path)
    
    # 提取时间和评分数据
    times = df['time'].values
    rates = df['rate'].values
    
    # 生成新的时间点（每2秒一个数据点，直到3000秒）
    new_times = np.arange(0, 2987, 2)
    
    # 创建新的评分数据列表
    new_rates = []
    current_rate = rates[0]
    current_time_index = 0
    
    # 遍历新的时间点
    for t in new_times:
        # 检查是否需要更新评分
        if current_time_index < len(times) - 1 and t >= times[current_time_index + 1]:
            current_time_index += 1
            current_rate = rates[current_time_index]
        new_rates.append(current_rate)
    
    # 创建新的DataFrame
    new_df = pd.DataFrame({'time': new_times, 'rate': new_rates})
    
    return new_df



In [3]:
base_dir = '/Users/li/Desktop/task-debate/behavdata/during_scan'

sub_list = [f'{x}' for x in range(13,51)]
run_list = ['1', '2', '3', '4', '5', '6']

for sub in sub_list:
    if sub == sub_list[0]:
        print('Doing …', sub, end ='\t')
    else:
        print(sub, end ='\t')
    
    sub_data = []
    for run in run_list:
        
        csv_file = os.path.join(base_dir, 'attitude_rate_during_scan', f'subject_{sub}_run_{run}_pos_time.csv')
        
        if run == '1':
            sub_run = pos_changes(csv_file)
        elif run == '2':
            sub_run = pos_changes(csv_file)
            sub_run = add_constant(sub_run, 500)
        elif run == '3':
            sub_run = pos_changes(csv_file)
            sub_run = add_constant(sub_run, 1098)
        elif run == '4':
            sub_run = pos_changes(csv_file)
            sub_run = add_constant(sub_run, 1484)
        elif run == '5':
            sub_run = pos_changes(csv_file)
            sub_run = add_constant(sub_run, 1892)
        elif run == '6':
            sub_run = pos_changes(csv_file)
            sub_run = add_constant(sub_run, 2464)

        sub_data = sub_data + sub_run
    
    df = pd.DataFrame(sub_data, columns =['rate','time'])
    df['time'] = df['time'].map(lambda x: '{:.3f}'.format(x))
    df['rate'] = df['rate'].astype(int)
    
    output_path = os.path.join(base_dir, 'combined_6runs', f'subject_{sub}_rate-time.csv')

    # df.to_csv(output_path, index=False)

Doing … 13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	

## Note: After each run, participants had 8 seconds to adjust their attitudes. Any attitude rating changes during this period were temporarily considered to occur at the final moment of the run. 
## In subsequent analyses focusing on attitude change points (e.g., predicting the direction of change or examining post-change increases in dACC activity), these points were excluded.

In [4]:
for sub in sub_list:
    
    if sub == '13':
        print('Doing sub', sub , end ='\t')
    else:
        print(sub, end ='\t')
        
    file_path = os.path.join(base_dir, 'combined_6runs', f'subject_{sub}_rate-time.csv')
    df = pd.read_csv(file_path)
    
    df.loc[(df['time'] > 500) & (df['time'] < 500+8), 'time'] = 500
    df.loc[(df['time'] > 1098) & (df['time'] < 1098+8), 'time'] = 1098
    df.loc[(df['time'] > 1484) & (df['time'] < 1484+8), 'time'] = 1484
    df.loc[(df['time'] > 1892) & (df['time'] < 1892+8), 'time'] = 1892
    df.loc[(df['time'] > 2464) & (df['time'] < 2464+8), 'time'] = 2464
    df.loc[(df['time'] > 2986) & (df['time'] < 2986+8), 'time'] = 2986
    
    output_path = os.path.join(base_dir, 'combined_6runs', f'subject_{sub}_rate-time.csv')
    
    # df.to_csv(output_path, index=False)

Doing sub 13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	

In [5]:
for sub in sub_list:
    file = os.path.join(base_dir, 'combined_6runs', f'subject_{sub}_rate-time.csv')
    time_rate = generate_rate_data_by_TR(file)
    output_file = os.path.join(base_dir, 'combined_6runs_per_TR', f'subject_{sub}_TR_rate.csv')
    # time_rate.to_csv(output_file, index=False)